Import required packages

In [92]:
import pandas as pd
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel

Read the file

In [93]:
file = pd.read_excel('shortcourses2566.xlsx')

Clean title with regular expresion

In [94]:
def clean_title(title):
    return re.sub("[^a-zA-Z0-9 ]", "", title)

Create dataframe of courses

In [95]:
content = file['หลักสูตรอบรมระยะสั้น'].drop_duplicates().fillna('')
df = pd.DataFrame(content)
content_clean = file['หลักสูตรอบรมระยะสั้น'].apply(clean_title).drop_duplicates().fillna('')
df_clean = pd.DataFrame(content_clean)

Create tfidf matrix

In [96]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(content_clean)

Create cosine similarities

In [97]:
cosine_similarities = linear_kernel(tfidf_matrix)

Create recommendation system function

In [166]:
def get_cosine_similarity_recommendations(column, value, limit):
    """Return a dataframe of content recommendations based on TF-IDF cosine similarity.
    
    Args:
        df (object): Pandas dataframe containing the text data. 
        column (string): Name of column used, i.e. 'title'. 
        value (string): Name of title to get recommendations for, i.e. 1982 Ferrari 308 GTSi For Sale by Auction
        cosine_similarities (array): Cosine similarities matrix from linear_kernel
        limit (int, optional): Optional limit on number of recommendations to return. 
        
    Returns: 
        Pandas dataframe. 
    """

    # Return indices for the target dataframe column and drop any duplicates
    indices = pd.Series(df[column]).drop_duplicates()
    
    # Get the index for the target value
    count = 0
    for name in indices:
        if name == value:
            break
        else:
            count = count + 1
    target_index = count

    # Get the cosine similarity scores for the target value
    cosine_similarity_scores = list(enumerate(cosine_similarities[target_index]))

    # Sort the cosine similarities in order of closest similarity
    cosine_similarity_scores = sorted(cosine_similarity_scores, key=lambda x: x[1], reverse=True)

    # Return tuple of the requested closest scores excluding the target item and index
    cosine_similarity_scores = cosine_similarity_scores[1:limit+1]

    # Extract the tuple values
    index = (x[0] for x in cosine_similarity_scores)
    scores = (x[1] for x in cosine_similarity_scores)

    # Get the indices for the closest items
    recommendation_indices = [i[0] for i in cosine_similarity_scores]

    # Get the actutal recommendations
    recommendations = df[column].iloc[recommendation_indices]

    # Return a recommendations
    recommendations = pd.DataFrame(list(zip(index, recommendations, scores)),
                      columns=['index','recommendation', 'cosine_similarity_score'])

    return recommendations

In [167]:
x = get_cosine_similarity_recommendations('หลักสูตรอบรมระยะสั้น',
                                      'การวิเคราะห์ข้อมูลอัจฉริยะด้วย RapidMiner (Intelligence Data analysis with RapidMiner) รุ่นที่ 2',
                                      10)
x

,index,recommendation,cosine_similarity_score
0,111,วัสดุวิศวกรรม (Engineering Materials) 2566 - ร...,0.415857
1,113,หุ่นยนต์วิจักษณ์ (Robotics Appreciation) 2566 ...,0.184917
2,48,นักวางแผนข้อมูลเมือง (City Data Planner : CDP)...,0.135525
3,129,เตรียมความพร้อมด้านดิจิทัลและทักษะภาษาอังกฤษ ส...,0.133195
4,52,การเตรียมความพร้อมในการคุ้มครองข้อมูลส่วนบุคคล...,0.128937
5,108,รังสีวิทยาวินิจฉัย,0.116179
6,0,หลักการและพื้นฐานของเครื่องมือทางรังสีวิทยา (B...,0.000000
7,1,การวินิจฉัยภาวะฉุกเฉินจากอุบัติเหตุ (Diagnosti...,0.000000
8,2,การวินิจฉัยภาวะฉุกเฉินที่ไม่ได้เกิดจากอุบัติเห...,0.000000
9,3,การพัฒนาปัญญาด้วยเกมต่อศัพท์ภาษาอังกฤษ (Intell...,0.000000


https://practicaldatascience.co.uk/data-science/how-to-create-content-recommendations-using-tf-idf